In [ ]:
import faiss
import numpy as np
from transformers import LlamaForCausalLM, LlamaTokenizer
from sentence_transformers import SentenceTransformer

# Загрузим модель для векторизации текстов
sentence_transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Пример кулинарных рецептов
recipes = [
    "Спагетти Болоньезе: отварить макароны, приготовить говяжий фарш с томатным соусом и чесноком",
    "Веганский салат: смешать салат, помидоры, огурцы и оливковое масло",
    "Куриное карри: приготовить курицу с порошком карри, луком, чесноком и кокосовым молоком",
]

In [ ]:
# Преобразуем рецепты в эмбеддинги
recipe_embeddings = sentence_transformer_model.encode(recipes)

# Создадим FAISS индекс
dimension = recipe_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(recipe_embeddings))

In [ ]:
# Пример запроса от пользователя
user_query = "Как приготовить веганский салат?"

# Преобразуем запрос в эмбеддинг
query_embedding = sentence_transformer_model.encode([user_query])

# Ищем ближайшие рецепты
D, I = index.search(np.array(query_embedding), k=2)

# Извлекаем наиболее релевантные рецепты
relevant_recipes = [recipes[i] for i in I[0]]
print("Relevant recipes:", relevant_recipes)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# Choose the appropriate model
model_name = "microsoft/Phi-3-mini-4k-instruct"  # or "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:

# Конкатенируем извлеченные рецепты с запросом пользователя
input_text = f"User query: {user_query}\nСоответствующие рецепты: {', '.join(relevant_recipes)}\nДайте четкий и лаконичный ответ:"

# Токенизация и генерация ответа
inputs = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs["input_ids"], max_length=150, num_return_sequences=1)

# Декодируем результат
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

In [ ]:
# Функция DeepSeek-R1 для оптимизации результатов поиска и генерации
def deepseek_r1_optimization(query, relevant_recipes):
    # Здесь может быть дополнительная логика для улучшения выбора ответа
    # Например, более точная фильтрация или уточнение запроса на основе DeepSeek-R1
    optimized_recipes = relevant_recipes  # Здесь может быть дополнительная логика для уточнения
    return optimized_recipes

# Преобразуем запрос в эмбеддинг
query_embedding = sentence_transformer_model.encode([user_query])

# Ищем ближайшие рецепты
D, I = index.search(np.array(query_embedding), k=2)

In [ ]:
# Извлекаем наиболее релевантные рецепты
relevant_recipes = [recipes[i] for i in I[0]]

# Применяем DeepSeek-R1 для оптимизации полученных рецептов
optimized_recipes = deepseek_r1_optimization(user_query, relevant_recipes)

# Генерация ответа с использованием Llama
def generate_response(query, relevant_recipes, tokenizer, model):
    input_text = f"User query: {user_query}\nСоответствующие рецепты: {', '.join(relevant_recipes)}\nДайте четкий и лаконичный ответ:"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(inputs["input_ids"], max_length=150, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Генерируем ответ
response = generate_response(user_query, optimized_recipes, tokenizer, model)
print(response)